### The goal of this assignment is to predict, whether the user browsing is Alice or The Intruder.

### The training data contains a column - target, where 1 - Alice, 0 - Intruder.

### I will attempt to solve this task using Logistic Regression to predict the target.

Import necessary libraries

In [1]:
from pathlib import Path
import pandas as pd

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler


Read data

In [22]:
FOLDER_PATH = Path('../../data/kaggle-competition')

X_train = pd.read_csv(FOLDER_PATH / 'train_sessions.csv')
X_test = pd.read_csv(FOLDER_PATH / 'test_sessions.csv')

In [23]:
X_train

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253556,253557,3474,2013-11-25 10:26:54,3474.0,2013-11-25 10:26:58,141.0,2013-11-25 10:27:03,2428.0,2013-11-25 10:27:04,106.0,...,2013-11-25 10:27:16,2428.0,2013-11-25 10:27:28,2428.0,2013-11-25 10:27:40,2428.0,2013-11-25 10:27:52,148.0,2013-11-25 10:27:53,0
253557,253558,12727,2013-03-12 16:01:15,12727.0,2013-03-12 16:01:16,2215.0,2013-03-12 16:01:16,38.0,2013-03-12 16:01:17,2215.0,...,2013-03-12 16:01:17,25444.0,2013-03-12 16:01:18,2215.0,2013-03-12 16:01:18,23.0,2013-03-12 16:01:18,21.0,2013-03-12 16:01:18,0
253558,253559,2661,2013-09-12 14:05:03,15004.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:05:10,5562.0,2013-09-12 14:06:29,5562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
253559,253560,812,2013-12-19 15:20:22,676.0,2013-12-19 15:20:22,814.0,2013-12-19 15:20:22,22.0,2013-12-19 15:20:22,39.0,...,2013-12-19 15:20:23,814.0,2013-12-19 15:20:23,570.0,2013-12-19 15:20:23,22.0,2013-12-19 15:20:24,570.0,2013-12-19 15:20:24,0


In [24]:
X_test

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,...,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
1,2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,...,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
2,3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,...,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
3,4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,...,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
4,5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,...,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82792,82793,812,2014-10-02 18:20:09,1039.0,2014-10-02 18:20:09,676.0,2014-10-02 18:20:09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82793,82794,300,2014-05-26 14:16:40,302.0,2014-05-26 14:16:41,302.0,2014-05-26 14:16:44,300.0,2014-05-26 14:16:44,300.0,...,1222.0,2014-05-26 14:17:19,302.0,2014-05-26 14:17:19,1218.0,2014-05-26 14:17:19,1221.0,2014-05-26 14:17:19,1216.0,2014-05-26 14:17:19
82794,82795,29,2014-05-02 11:21:56,33.0,2014-05-02 11:21:56,35.0,2014-05-02 11:21:56,22.0,2014-05-02 11:22:03,37.0,...,6779.0,2014-05-02 11:22:03,30.0,2014-05-02 11:22:03,21.0,2014-05-02 11:22:04,23.0,2014-05-02 11:22:04,6780.0,2014-05-02 11:22:04
82795,82796,5828,2014-05-03 10:05:25,23.0,2014-05-03 10:05:27,21.0,2014-05-03 10:05:27,804.0,2014-05-03 10:05:27,21.0,...,3350.0,2014-05-03 10:05:37,23.0,2014-05-03 10:05:37,894.0,2014-05-03 10:05:38,21.0,2014-05-03 10:05:38,961.0,2014-05-03 10:05:38


In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253561 entries, 0 to 253560
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  253561 non-null  int64  
 1   site1       253561 non-null  int64  
 2   time1       253561 non-null  object 
 3   site2       250098 non-null  float64
 4   time2       250098 non-null  object 
 5   site3       246919 non-null  float64
 6   time3       246919 non-null  object 
 7   site4       244321 non-null  float64
 8   time4       244321 non-null  object 
 9   site5       241829 non-null  float64
 10  time5       241829 non-null  object 
 11  site6       239495 non-null  float64
 12  time6       239495 non-null  object 
 13  site7       237297 non-null  float64
 14  time7       237297 non-null  object 
 15  site8       235224 non-null  float64
 16  time8       235224 non-null  object 
 17  site9       233084 non-null  float64
 18  time9       233084 non-null  object 
 19  si

Gather columns for time/site. Convert time columns from string to datetime so that operations could be performed.

In [26]:
time_cols = [col for col in X_train.columns if col.startswith("time")]
site_cols = [col for col in X_train.columns if col.startswith("site")]
ids = X_test["session_id"].astype('int').copy()

for col in time_cols:
    X_train[col] = pd.to_datetime(X_train[col], errors='coerce')
    X_test[col] = pd.to_datetime(X_test[col], errors='coerce')

Since time column can be null, find the last time that was not null

In [27]:
def get_last_valid_time(row):
    for i in range(10, 0, -1):
        cell = row[f"time{i}"]
        if pd.notnull(cell):
            return cell

    return pd.NaT

In [28]:
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

Lets extract the most important features from the timestamps

In [29]:
def engineer_features(df):
    df["last_time"] = df.apply(get_last_valid_time, axis=1)
    df["duration"] = (df["last_time"] - df["time1"]).dt.total_seconds().fillna(0)
    df["day_of_week"] = df["time1"].dt.dayofweek
    df["start_hour"] = df["time1"].dt.hour
    df["end_hour"] = df["last_time"].dt.hour
    df["time_of_day"] = df["start_hour"].apply(time_of_day)

    df.drop("last_time", axis=1, inplace=True)
    df.drop(time_cols, axis=1, inplace=True)

    df = pd.get_dummies(df, columns=["time_of_day"])

    return df

Inorder for the logistic regression model to not favor some higher value feature, they need to be balanced/scaled by StandardScaler.

In [30]:
def scale_numeric_features(numeric_features, df_train, df_test):
    scaler = StandardScaler()

    df_train[numeric_features] = scaler.fit_transform(df_train[numeric_features])
    df_test[numeric_features] = scaler.transform(df_test[numeric_features])

    return df_train, df_test

Seperate data for fitting in the models

In [31]:
y_train = X_train["target"]
X_train.drop("target", axis=1, inplace=True)

In [32]:
X_train = engineer_features(X_train)
X_test = engineer_features(X_test)

numeric_features = ["duration", "day_of_week", "start_hour", "end_hour"]

X_train, X_test = scale_numeric_features(numeric_features, X_train, X_test)

Tfidf vectorizer helps to segregate the visited sites. Unique sites - more importance, common sites - less importance.

In [33]:
def handle_site_features(X_train, X_test, site_features):
    X_train_sites = X_train[site_features].astype(str).agg(' '.join, axis=1)
    X_test_sites = X_test[site_features].astype(str).agg(' '.join, axis=1)

    tfidf = TfidfVectorizer(analyzer='word', token_pattern=r'\S+', ngram_range=(1, 3), max_features=30000)
    X_train_tfidf = tfidf.fit_transform(X_train_sites)
    X_test_tfidf = tfidf.transform(X_test_sites)

    return X_train_tfidf, X_test_tfidf

In [34]:
X_train_tfidf, X_test_tfidf = handle_site_features(X_train, X_test, site_cols)

X_train = X_train.drop(columns=site_cols)
X_test = X_test.drop(columns=site_cols)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

Combine two sets of features (TF-IDF vectorizer and our custom time features) into a matrix.

In [35]:
X_train_combined = hstack([X_train_tfidf, X_train.values])
X_test_combined = hstack([X_test_tfidf, X_test.values])

Use StratifiedKFold for even distribution of "Alice" and "Intruder" classes in each fold.

In [36]:
stratfold = StratifiedKFold(shuffle=True, random_state=1)

Train Logistic Regression Cross-Validation model with custom parameters:
1. Cs - 10 values from 1e-4 and 1e+4
2. Cv - custom fold algorithm
3. Scoring - "roc_auc", since the task requires this output
4. Solver - "lbfgs", uses L2 regularization by default, to keep all the features

In [45]:
logreg = LogisticRegressionCV(
    Cs=10,
    cv=stratfold,
    scoring="roc_auc",
    solver="lbfgs",
    random_state=1,
    n_jobs=-1,
    max_iter=5000
)

In [46]:
logreg.fit(X_train_combined, y_train)

,Cs,10
,fit_intercept,True
,cv,StratifiedKFo... shuffle=True)
,dual,False
,penalty,'l2'
,scoring,'roc_auc'
,solver,'lbfgs'
,tol,0.0001
,max_iter,5000
,class_weight,None
,n_jobs,-1


In [47]:
cv_scores = cross_val_score(logreg, X_train_combined, y_train, cv=3, scoring='roc_auc')
print(f"Mean AUC: {cv_scores.mean():.4f}, Std: {cv_scores.std():.4f}")

Mean AUC: 0.9476, Std: 0.0123


Output to CSV file for submission

In [48]:
y_pred = logreg.predict_proba(X_test_combined)[:, 1]

result_df = pd.DataFrame({
    'session_id': ids,
    'target': y_pred
})

print(result_df.shape)

result_df.to_csv('submission.csv', index=False)


(82797, 2)


92,3 % Test score achieved!